In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [2]:
delivery = pd.read_csv(r'D:\Data Science\Projects\Cricket t20\Data\Raw\source1\Every_ball_data.csv')
match = pd.read_csv(r'D:\Data Science\Projects\Cricket t20\Data\Raw\source1\T20_international_matches.csv')

In [4]:
match.head()

,city,date,gender,match_referees,tv_umpires,first_umpire,second_umpire,win_by,win_with,result,player_of_match,team_1,team_2,elected_first,toss_winner,venue,mergeid
0,Southampton,2005-06-13,male,JJ Crowe,MR Benson,NJ Llong,JW Lloyds,runs,100.0,England,['KP Pietersen'],England,Australia,bat,England,The Rose Bowl,31231
1,Auckland,2005-02-17,male,CH Lloyd,DB Cowie,BF Bowden,AL Hill,runs,44.0,Australia,['RT Ponting'],New Zealand,Australia,bat,Australia,Eden Park,32232
2,Johannesburg,2005-10-21,male,BC Broad,KH Hurter,IL Howell,BG Jerling,wickets,5.0,New Zealand,['JS Patel'],South Africa,New Zealand,field,New Zealand,New Wanderers Stadium,33233
3,Bristol,2006-08-28,male,MJ Procter,IJ Gould,PJ Hartley,NJ Llong,wickets,5.0,Pakistan,['Shahid Afridi'],England,Pakistan,bat,England,County Ground,34234
4,Southampton,2006-06-15,male,CH Lloyd,PJ Hartley,IJ Gould,NJ Llong,runs,2.0,Sri Lanka,['ST Jayasuriya'],England,Sri Lanka,bat,Sri Lanka,The Rose Bowl,35235


In [5]:
match['city'] = match['city'].fillna(match['venue'])

delivery.rename(columns={'team_name':'batting_team'},inplace=True)
match.rename(columns={'result':'winner'},inplace=True)

In [6]:
delivery.head(10)

,inning,batting_team,over_num,batsman_name,bowler,non_striker,extra_type,extra_run,total_run,batsman_run,dismissal_kind,dismissed_player,fielders_name,mergeid
0,1,England,0,ME Trescothick,B Lee,GO Jones,NaN,0,0,0,NaN,NaN,NaN,31231
1,1,England,0,ME Trescothick,B Lee,GO Jones,NaN,0,1,1,NaN,NaN,NaN,31231
2,1,England,0,GO Jones,B Lee,ME Trescothick,NaN,0,0,0,NaN,NaN,NaN,31231
3,1,England,0,GO Jones,B Lee,ME Trescothick,NaN,0,0,0,NaN,NaN,NaN,31231
4,1,England,0,GO Jones,B Lee,ME Trescothick,NaN,0,0,0,NaN,NaN,NaN,31231
5,1,England,0,GO Jones,B Lee,ME Trescothick,noballs,1,1,0,NaN,NaN,NaN,31231
6,1,England,0,GO Jones,B Lee,ME Trescothick,NaN,0,2,2,NaN,NaN,NaN,31231
7,1,England,1,ME Trescothick,GD McGrath,GO Jones,NaN,0,0,0,NaN,NaN,NaN,31231
8,1,England,1,ME Trescothick,GD McGrath,GO Jones,NaN,0,0,0,NaN,NaN,NaN,31231
9,1,England,1,ME Trescothick,GD McGrath,GO Jones,wides,1,1,0,NaN,NaN,NaN,31231


In [7]:
delivery.shape

(507721, 14)

In [8]:
delivery.shape

(507721, 14)

In [9]:
delivery['extra_type'].unique()

array([nan, 'noballs', 'wides', 'legbyes', 'byes', 'penalty'],
      dtype=object)

In [10]:
delivery['mergeid'].unique()

array([      31231,       32232,       33233, ..., 32222232222,
       32223232223, 32224232224], dtype=int64)

In [11]:
#Create a column for number of balls

for each_match in tqdm(delivery['mergeid'].unique()):
    macth_df = delivery[delivery['mergeid']==each_match]
    
    for each_inning in macth_df['inning'].unique():
        inning_df = macth_df[macth_df['inning']==each_inning]

        for each_inning in inning_df['inning'].unique():
            nums = 0
            for i, row in inning_df.iterrows():
                if nums < 6:
                    if str(inning_df.loc[i,'extra_type']) not in ['noballs','wides']:
                        delivery.loc[i,'balls']= nums + 1
                        nums = nums + 1
                    else:
                        delivery.loc[i,'balls']= nums+1
                        nums = nums
                else:
                    if str(delivery.loc[i,'extra_type']) not in ['noballs','wides']:
                        nums = 0
                        delivery.loc[i,'balls']= nums + 1
                        nums = 1
                    else:
                        nums = 0
                        delivery.loc[i,'balls']= nums+1
                        nums = nums 

  0%|          | 0/2224 [00:00<?, ?it/s]

In [12]:
delivery['balls'] = delivery['balls'].astype(int)

delivery['over_num'] = delivery['over_num'] + 1

In [13]:
total_score_df = delivery.groupby(['mergeid','inning']).sum()['total_run'].reset_index()

total_score_df = total_score_df[total_score_df['inning']==1]

C:\Users\acer\AppData\Local\Temp\ipykernel_2100\1448408146.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  total_score_df = delivery.groupby(['mergeid','inning']).sum()['total_run'].reset_index()


In [14]:
total_score_df.head()

,mergeid,inning,total_run
0,31231,1,179
2,32232,1,214
4,33233,1,133
6,34234,1,144
8,35235,1,163


In [15]:
total_score_df[total_score_df['mergeid'] == 385423854]

,mergeid,inning,total_run
1712,385423854,1,158


In [16]:
#change the score and update the DRS score
total_score_df.loc[total_score_df['mergeid'] == 385423854,'total_run'] = 174

In [17]:
total_score_df[total_score_df['mergeid'] == 385423854]

,mergeid,inning,total_run
1712,385423854,1,174


In [18]:
#create combinations of both the teams
all_matches = delivery[['mergeid','batting_team']].drop_duplicates()
all_matches2 = all_matches.copy()
all_matches2.rename(columns={'batting_team':'bowling_team'},inplace=True)

all_matches3 = pd.merge(all_matches, all_matches2, on = 'mergeid', how= 'left')

all_matches3 = all_matches3[all_matches3['batting_team']!=all_matches3['bowling_team']]

In [19]:
all_matches3.head()

,mergeid,batting_team,bowling_team
1,31231,England,Australia
2,31231,Australia,England
5,32232,Australia,New Zealand
6,32232,New Zealand,Australia
9,33233,South Africa,New Zealand


In [20]:
delivery.head()

,inning,batting_team,over_num,batsman_name,bowler,non_striker,extra_type,extra_run,total_run,batsman_run,dismissal_kind,dismissed_player,fielders_name,mergeid,balls
0,1,England,1,ME Trescothick,B Lee,GO Jones,NaN,0,0,0,NaN,NaN,NaN,31231,1
1,1,England,1,ME Trescothick,B Lee,GO Jones,NaN,0,1,1,NaN,NaN,NaN,31231,2
2,1,England,1,GO Jones,B Lee,ME Trescothick,NaN,0,0,0,NaN,NaN,NaN,31231,3
3,1,England,1,GO Jones,B Lee,ME Trescothick,NaN,0,0,0,NaN,NaN,NaN,31231,4
4,1,England,1,GO Jones,B Lee,ME Trescothick,NaN,0,0,0,NaN,NaN,NaN,31231,5


In [21]:
delivery = pd.merge(delivery,all_matches3, on = ['mergeid','batting_team'], how ='left')

In [22]:
delivery.shape

(507721, 16)

In [23]:
match_df = pd.merge(match, total_score_df[['mergeid','total_run']], on = 'mergeid',how='left')

match_df = match_df[['mergeid','city','winner','total_run','venue','gender','date']]


In [24]:
match_df.head(2)

,mergeid,city,winner,total_run,venue,gender,date
0,31231,Southampton,England,179,The Rose Bowl,male,2005-06-13
1,32232,Auckland,Australia,214,Eden Park,male,2005-02-17


In [25]:
delivery_df = pd.merge(match_df, delivery, on = 'mergeid', how='left' ) 

In [26]:
delivery_df = delivery_df[delivery_df['inning']==2]
delivery_df = delivery_df[delivery_df['gender']=='male']

In [27]:
delivery_df.head()

,mergeid,city,winner,total_run_x,venue,gender,date,inning,batting_team,over_num,...,non_striker,extra_type,extra_run,total_run_y,batsman_run,dismissal_kind,dismissed_player,fielders_name,balls,bowling_team
125,31231,Southampton,England,179,The Rose Bowl,male,2005-06-13,2,Australia,1,...,ML Hayden,NaN,0,0,0,NaN,NaN,NaN,1,England
126,31231,Southampton,England,179,The Rose Bowl,male,2005-06-13,2,Australia,1,...,ML Hayden,NaN,0,4,4,NaN,NaN,NaN,2,England
127,31231,Southampton,England,179,The Rose Bowl,male,2005-06-13,2,Australia,1,...,ML Hayden,NaN,0,0,0,NaN,NaN,NaN,3,England
128,31231,Southampton,England,179,The Rose Bowl,male,2005-06-13,2,Australia,1,...,ML Hayden,NaN,0,0,0,NaN,NaN,NaN,4,England
129,31231,Southampton,England,179,The Rose Bowl,male,2005-06-13,2,Australia,1,...,ML Hayden,NaN,0,4,4,NaN,NaN,NaN,5,England


In [28]:
delivery_df['current_score'] = delivery_df.groupby('mergeid').cumsum()['total_run_y']

C:\Users\acer\AppData\Local\Temp\ipykernel_2100\1718743870.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  delivery_df['current_score'] = delivery_df.groupby('mergeid').cumsum()['total_run_y']


In [29]:
delivery_df.head(2)

,mergeid,city,winner,total_run_x,venue,gender,date,inning,batting_team,over_num,...,extra_type,extra_run,total_run_y,batsman_run,dismissal_kind,dismissed_player,fielders_name,balls,bowling_team,current_score
125,31231,Southampton,England,179,The Rose Bowl,male,2005-06-13,2,Australia,1,...,NaN,0,0,0,NaN,NaN,NaN,1,England,0
126,31231,Southampton,England,179,The Rose Bowl,male,2005-06-13,2,Australia,1,...,NaN,0,4,4,NaN,NaN,NaN,2,England,4


In [30]:
delivery_df['runs_left'] = delivery_df['total_run_x'] - delivery_df['current_score']

In [31]:
delivery_df['runs_left'] = delivery_df['runs_left'].apply(lambda x: x+1)
delivery_df['runs_left'] = np.where(delivery_df['runs_left']<0,0,delivery_df['runs_left'])

delivery_df['balls_left'] = 126 - (delivery_df['over_num']*6 + delivery_df['balls'])
delivery_df['dismissed_player'] = np.where(delivery_df['dismissal_kind'].isnull(),0,1)

delivery_df['dismissed_player'] = delivery_df['dismissed_player'].astype(int)

wickets = delivery_df.groupby('mergeid').cumsum()['dismissed_player'].values

delivery_df['wickets'] = 10 - wickets

#current RR
delivery_df['crr'] = (delivery_df['current_score']*6)/(120-delivery_df['balls_left'])

#required RR
delivery_df['rrr'] = (delivery_df['runs_left']*6)/delivery_df['balls_left']
delivery_df['rrr'] = delivery_df['rrr'].fillna(0)
delivery_df['rrr'] = np.where(delivery_df['rrr'].astype(str) == 'inf',111,delivery_df['rrr'])

delivery_df['result'] = delivery_df.apply(lambda x: 1 if x['winner'] == x['batting_team'] else 0,axis=1)



delivery_df['batting_team'] = delivery_df['batting_team'].str.upper()
delivery_df['bowling_team'] = delivery_df['bowling_team'].str.upper()
delivery_df['city'] = delivery_df['city'].str.upper()

C:\Users\acer\AppData\Local\Temp\ipykernel_2100\1924953745.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.cumsum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  wickets = delivery_df.groupby('mergeid').cumsum()['dismissed_player'].values


In [32]:
delivery_df[(delivery_df['mergeid'] == 35235) & (delivery_df['over_num'] == 20)].tail(2)

,mergeid,city,winner,total_run_x,venue,gender,date,inning,batting_team,over_num,...,fielders_name,balls,bowling_team,current_score,runs_left,balls_left,wickets,crr,rrr,result
1182,35235,SOUTHAMPTON,Sri Lanka,163,The Rose Bowl,male,2006-06-15,2,ENGLAND,20,...,NaN,5,SRI LANKA,159,5,1,5,8.016807,30.0,0
1183,35235,SOUTHAMPTON,Sri Lanka,163,The Rose Bowl,male,2006-06-15,2,ENGLAND,20,...,NaN,6,SRI LANKA,161,3,0,5,8.050000,111.0,0


In [33]:
delivery_df.head()

,mergeid,city,winner,total_run_x,venue,gender,date,inning,batting_team,over_num,...,fielders_name,balls,bowling_team,current_score,runs_left,balls_left,wickets,crr,rrr,result
125,31231,SOUTHAMPTON,England,179,The Rose Bowl,male,2005-06-13,2,AUSTRALIA,1,...,NaN,1,ENGLAND,0,180,119,10,0.0,9.075630,0
126,31231,SOUTHAMPTON,England,179,The Rose Bowl,male,2005-06-13,2,AUSTRALIA,1,...,NaN,2,ENGLAND,4,176,118,10,12.0,8.949153,0
127,31231,SOUTHAMPTON,England,179,The Rose Bowl,male,2005-06-13,2,AUSTRALIA,1,...,NaN,3,ENGLAND,4,176,117,10,8.0,9.025641,0
128,31231,SOUTHAMPTON,England,179,The Rose Bowl,male,2005-06-13,2,AUSTRALIA,1,...,NaN,4,ENGLAND,4,176,116,10,6.0,9.103448,0
129,31231,SOUTHAMPTON,England,179,The Rose Bowl,male,2005-06-13,2,AUSTRALIA,1,...,NaN,5,ENGLAND,8,172,115,10,9.6,8.973913,0


In [34]:
delivery_df[(delivery_df['balls_left']==6) & (delivery_df['runs_left']>=28)].head()

,mergeid,city,winner,total_run_x,venue,gender,date,inning,batting_team,over_num,...,fielders_name,balls,bowling_team,current_score,runs_left,balls_left,wickets,crr,rrr,result
453,32232,AUCKLAND,Australia,214,Eden Park,male,2005-02-17,2,NEW ZEALAND,19,...,NaN,6,AUSTRALIA,161,54,6,2,8.473684,54.0,0
2162,39239,SYDNEY CRICKET GROUND,Australia,221,Sydney Cricket Ground,male,2007-01-09,2,ENGLAND,19,...,NaN,6,AUSTRALIA,131,91,6,2,6.894737,91.0,0
3958,3172317,DURBAN,Pakistan,171,Kingsmead,male,2007-09-12,2,SCOTLAND,19,...,NaN,6,PAKISTAN,114,58,6,1,6.000000,58.0,0
4686,3202320,CAPE TOWN,England,188,Newlands,male,2007-09-13,2,ZIMBABWE,19,...,NaN,6,ENGLAND,135,54,6,4,7.105263,54.0,0
4939,3212321,JOHANNESBURG,Sri Lanka,260,New Wanderers Stadium,male,2007-09-14,2,KENYA,19,...,NaN,6,SRI LANKA,86,175,6,2,4.526316,175.0,0


In [35]:
delivery_df[(delivery_df['balls_left']==6) & (delivery_df['runs_left']>=28)]['mergeid'].nunique()

297

In [36]:
delivery_df.to_csv(r'D:\Data Science\Projects\Cricket t20\Data\Prepared\prepared.csv',index=False)